In [ ]:
'''
TODO: See what state each 
Make a report
'''

In [112]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import calendar
import numpy as np
import os
import datetime

In [113]:
'''
"nodalData" is the path to the file containing the hourly aggregated load
data for Mexico.

"fileName" is a dictionary for the path to the hourly Southern California
Edison data for each category.

"catName" is a dictionary for the name of each category in the SENER data
named "fullusage". These names are in Spanish using only utf-8 characters
'''


nodalData = "HourlyLoadPerNode.csv"

fileName = {}
fileName["MedInd"] = "Industrial Data/MedIndPros.csv"
fileName["LarInd"] = "Industrial Data/LargeIndPros.csv"
fileName["Agri"] = "Agriculture Data/AgriPros.csv"

catName = {}
catName["MedInd"] = "Mediana Industria"
catName["LarInd"] = "Gran Industria"
catName["Agri"] = "Agr?cola"

daysInMonth = [31,28,31,30,31,30,31,31,30,31,30,31]

categories = ["MedInd","LarInd","Agri"]

In [114]:
'''
The dictionary "time" is used to map each state to a time shift to apply
to the final load. Since the "HourlyLoadPerNode" data is set to standard
central time (UTC-6), this shift is calculated by using the (UTC-T) value
from the desired state and calculating T-6

The shift function is a method of applying this shift to a list cyclically
'''

timezones = pd.read_csv("TimeZones.csv")
states = timezones["state"]
utc = timezones["utc"]

time = {}
for i in range(0,len(states)):
    time[states[i]] = utc[i] - 6
    
    
def shift(e,ind):
    """
        Args:
            e (list): Hourly load curve
            ind (int): Index to shift the hourly load curve by
        
        Return:
            list: Shifted hourly load curve
    """
    ans = []
    for i in range(0,len(e)):
        ans.append(e[(i+ind)%len(e)])
    return ans

In [115]:
'''
This cell is primarily for reading the input

"nodal" is a DataFrame for the "HourlyLoadPerNode" data

"df" is a DataFrame for the SENER "fullusage" data

"date" is a dictionary used to translate date (monthly) strings (i.e 06-2018)
into integer indexes which are more convenient to use 

"data" is a dictionary to store all the monthly data from the SENER 

"fullusage" data in a more conviently accessed method. Calling 
data[NameOfState][NameOfCategory] returns a list with the monthly load
profile for that state within a given category. To access a specific
month, call data[NameOfState][NameOfCategory][date["MONTH"]]
'''

nodal = pd.read_csv(nodalData)

#Reading Input

df = pd.read_csv("General Data/fullusage.csv", encoding = 'utf-8')

#Reads all dates from fullusage.csv and assigns them to an integer in the date dictionary
date = {}
rdate = df["Descripcion"].as_matrix()
for i in range(0,len(rdate)):
    date[rdate[i]] = i
    
#Reads the data with data being stored as data{state}{category}[monthly load profile]
data = {}
for c in df:
    #Condition if column name is a category
    if(c[0:8] == " " * 8):
        data[state][c.strip().split(".")[0]] = df[c].as_matrix()
    #Condition if column name is the name of a state
    elif(c[0:4] == " " * 4):
        state = c.strip().lower()
        data[state] = {}
        data[state]["Total"] = df[c].as_matrix()

In [116]:
'''
Node Class contains all the methods to simulate the load curve for a Mexican Region
'''
class node:
    
    loads = {}
    
    def calcPerOfSec(state, month, cat):
        """
            Args:
                state (String):
                month (int): Index of month in full usage
                cat (String):
            
            Returns:
                float: Percent of State's total load curve used in a specific category
        """
        categoryUsage = data[state][catName[cat]][month]
        totalUsage = data[state]["Total"][month]
        return float(categoryUsage)/float(totalUsage)
    
    def calcTotal(region, month,year):
        """
            Args:
                region (String): Name of region as shown on "HourlyLoadPerNode.csv" data
                month (int): Month of the Year
                year (int): Year being simulated
            
            Returns:
                float: Aggregated monthly load
        """
        r0 = nodal["year"].as_matrix();
        r1 = nodal["month"].as_matrix();
        r2 = nodal[region].as_matrix();
        
        ans = 0
        
        x = True
        for i in range(0,len(r1)):
            if(str(r1[i]) == str(month) and str(year) == str(r0[i])):
                ans += r2[i]
                x = False
            if(x == False and r1[i] != month):
                break
                
        return ans
            
    def create(self, name, state):
        """
            Args:
                name (String): Name of electricity provider
                state (String): State which contains the electricity provider
            
            Returns:
                void
        """
        self.name = name
        self.state = state

    def loadCurve(self,cat,start,data):
        """
            Args:
                cat (String): Category being simulated
                start (int): Year to start
                data (list): Monthly load curve
                
            Returns:
                list: Hourly load curve
        """
        ans = []
        temp = pd.read_csv(fileName[cat])
        TOU = []
        for column in temp:
            if("/" in column):
                TOU.append(temp[column].as_matrix())
        day = calendar.weekday(start,1,1)
        cur = (day-3+7)%7
        
        
        for j in range(0,len(data)):
            for k in range(0,daysInMonth[j%12]):
                for l in range(0,24):
                    ans.append(data[int(j)] * TOU[int(cur)][int(l)])
                cur += 1
                cur %= 365

        return ans

    def generate(self, year, write = False):
        """
            Args:
                year (int): year that you want to simulate
                write (bool): Whether you would like the generated load curve to be printed to a csv file
                
            Returns:
                list: Hourly load curve for given year
        """
        if(os.path.exists("Generated/" + self.name + " " + str(year)  + ".csv")):
            return
        start = "2008-01"
        months = 12
        for cat in categories:
            d = []
            for i in range(date[start],date[start] + months):
                curmonth = int(start.split("-")[1])
                temp1 = node.calcPerOfSec(self.state,i,cat) * node.calcTotal(self.id,curmonth,year)
                if(temp1 != "N/D"):
                    d.append(float(temp1))
                else:
                    d.append(0)
            self.loads[cat] = self.loadCurve(cat,year,d)
            tz = time[self.state.replace(" ","_")]
            self.loads[cat] = shift(self.loads[cat],tz)
        
        full = self.aggregate()
        
        if(write):
            w = pd.DataFrame()
            for b in self.loads:
                w[b] = self.loads[b]
            l = pd.read_csv(nodalData)[name].as_matrix()
            f = []
            df = pd.read_csv("HourlyLoadPerNode.csv")
            y = df["year"].as_matrix()
            
            for i in range(0,len(y)):
                if(str(y[i]) == str(year)):
                    f.append(l[i])
            
            
            res = []
            for i in range(0,len(full)):
                res.append(max(0,f[i] - full[i]))
            w["Residential"] = res
            w["Full"] = f[0:len(res)]
            w.to_csv("Generated/" + self.name + " " + str(year)  + ".csv")
            
    def aggregate(self):
        """
            Args:
                N/A
                
            Returns:
                list: Hourly aggregated load curve
        """
        ans = []
        for b in self.loads:
            if(len(ans) == 0):
                ans = [0] * len(self.loads[b])
            ans = np.add(ans,self.loads[b])
        return ans

In [117]:
st2 = {}
df = pd.read_csv("nodelog2.csv")
for i in range(0,len(df["STATE"])):
    #read each state and its corresponding regions
    state = df["STATE"].iloc[i].lower()
    region = df["REGION"].iloc[i].lower()
   
    if(not(region in st2)):
        st2[region] = []
    st2[region].append(state)

for i in st2:
    st2[i] = list(set(st2[i]))
    

In [118]:
for c in nodal:
    if "-" in c:
        try:
            print()
            print(c)
            city = c.split("-")[1].replace("_"," ")
            state = st2[city][0]
            if state == 'nayarit':
                state = 'jalisco'
            name = c
            n = node()
            n.create(city,state)
            n.id = name
            n.generate(2016,write = True)
            print("GENERATED")
        except KeyError as e:
            print("KEY ERROR in " + str(c))
            print(e)
            print(name + " " + state + " " + city)
            print()
        except ValueError as e:
            print("VALUE ERROR in " + str(c))
            print(e)
            print(name + " " + state + " " + city)
            print()
        


01-hermosillo
GENERATED

02-cananea
GENERATED

03-obregon
GENERATED

04-los_mochis
GENERATED

05-culiacan
GENERATED

06-mazatlan
GENERATED

07-juarez
GENERATED

08-moctezuma
GENERATED

09-chihuahua
GENERATED

10-durango
GENERATED

11-laguna
GENERATED

12-rio_escondido
GENERATED

13-nuevo_laredo
GENERATED

14-reynosa
GENERATED

15-matamoros
GENERATED

16-monterrey
GENERATED

17-saltillo
GENERATED

18-valles
GENERATED

19-huasteca
GENERATED

20-tamazunchale
GENERATED

21-guemez
GENERATED

22-tepic
GENERATED

23-guadalajara
GENERATED

24-aguascalientes
GENERATED

25-san_luis_potosi
GENERATED

26-salamanca
GENERATED

27-manzanillo
GENERATED

28-carapan
GENERATED

29-lazaro_cardenas
GENERATED

30-queretaro
GENERATED

31-central
GENERATED

32-poza_rica
GENERATED

33-veracruz
GENERATED

34-puebla
GENERATED

35-acapulco
GENERATED

36-temascal
GENERATED

37-coatzacoalcos
GENERATED

38-tabasco
GENERATED

39-grijalva
VALUE ERROR in 39-grijalva
could not convert string to float: 'N/D'
39-grijalva

In [119]:
pd.DataFrame.from_dict(data=st2, orient='index').to_csv("States.csv")

PermissionError: [Errno 13] Permission denied: 'States.csv'

In [ ]:
print(data)